In [ ]:
import re
import json
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
dataset_name = 'webqsp'
split = 'train'


with open(f"./../data/preprocessed_data/{dataset_name}/_domains/total/gpt_labeled_{dataset_name}_raw_{split}.jsonl") as f:
    data = [json.loads(line) for line in f]
print(data.keys())

In [ ]:
# from strings to tuples of triplets
def get_tuples(input):
    trunc_cnt = 0
    pattern = r'(.*?)(,.*?\..*?\..*?,)(.*)'

    output_evi = []
    for evi in input:
        match = re.search(pattern, evi)
        if match is None or match.lastindex < 3:
            trunc_cnt += 1
        else:
            output_evi.append((match.group(1).replace('(', '').strip(), match.group(2)[1:-1].strip(), match.group(3).replace(')', '').strip()))
    return output_evi

In [ ]:
non_exist_cnt = 0
total_cnt = 0
pattern = r'(.*?)(,.*?\..*?\..*?,)(.*)'
num_evi = []
for idx, each in enumerate(tqdm(data)):
    input_evi = each['user_query'].split('Question')[0].split('\n')[1:-2]
    input_evi = get_tuples(input_evi)

    output = each['response'].split('\n')
    output = [evi for evi in output if 'evidence:' in evi]
    output = [evi[evi.find('evidence:') + len('evidence:') + 1 :].strip() for evi in output]

    output_evi = get_tuples(output)
    good_evi = []
    for each_evi in output_evi:
        total_cnt += 1

        # two directions both count. in some cases, gpt will correct the order
        if (each_evi in input_evi or tuple(reversed(each_evi)) in input_evi):
            good_evi.append(each_evi)
        else:
            non_exist_cnt += 1
            # if non_exist_cnt == 100:
                # raise ValueError(f'{each_evi} not in {input_evi}')

    data[idx]['gpt_labeled_evi'] = good_evi
    num_evi.append(len(output_evi))

print(non_exist_cnt, total_cnt, non_exist_cnt / total_cnt)

In [ ]:
# for samples GPT provides more than 20 evidences, perhaps remove them
for idx, each in enumerate(tqdm(data)):
    if len(data[idx]['gpt_labeled_evi']) > 20:
        data[idx]['gpt_labeled_evi'] = []
fig = plt.hist(num_evi, bins=range(0, 50))

In [ ]:
# count coverage of samples that GPT provides valid evidences
cnt = 0
for each in data:
    if len(each['gpt_labeled_evi']) > 0:
        cnt += 1
print(cnt, len(data), cnt / len(data))

In [ ]:
res = {}

for each in data:
    if len(each['gpt_labeled_evi']) > 0:
        if "triples" in each:
            res[each['triples']] = each['gpt_labeled_evi']
        elif "qid" in each:
            res[each['qid']] = each['gpt_labeled_evi']

In [ ]:
torch.save(res, f'./../data/preprocessed_data/{dataset_name}/_domains/total/gpt_labeled_{dataset_name}_cleaned_{split}.pt')